In [ ]:
#鉴别图片是否包含不正当内容,用户上传图片文件到特定存储桶后,rekognition会检测图片.并返回Revealing Clothes,Suggestive,Confidence等.

In [ ]:
#相关服务参考https://docs.aws.amazon.com/zh_cn/rekognition/latest/dg/procedure-moderate-images.html

In [ ]:
# 0.初始化资源

In [6]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
bucketname = "bucketname" #不需要S3://

In [ ]:
# 1.创建相关角色

In [ ]:
%%bash
tee trust-policy.json << END
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
END

In [ ]:
!aws iam create-role --role-name lambda-ex --assume-role-policy-document file://trust-policy.json

In [9]:
%%bash
aws iam attach-role-policy --role-name lambda-ex --policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole
aws iam attach-role-policy --role-name lambda-ex --policy-arn arn:aws:iam::aws:policy/AmazonRekognitionFullAccess
aws iam attach-role-policy --role-name lambda-ex --policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess

In [ ]:
# 2. 部署lambda代码

In [10]:
!zip function.zip lambda_function.py

  adding: lambda_function.py (deflated 61%)


In [ ]:
!aws lambda create-function --function-name DetectModeration \
--zip-file fileb://function.zip --handler lambda_function.lambda_handler --runtime python3.10 \
--role arn:aws:iam::$account_id:role/lambda-ex

In [165]:
!aws lambda list-functions --query "Functions[?FunctionName=='DetectModeration'].FunctionArn" --output text

arn:aws:lambda:ap-northeast-1:544592066775:function:DetectModeration


In [ ]:
!aws lambda add-permission \
  --function-name DetectModeration \
  --action "lambda:InvokeFunction" \
  --principal s3.amazonaws.com \
  --source-arn arn:aws:s3:::$bucketname \
  --statement-id s3-trigger

In [167]:
import json
import boto3

lambda_client = boto3.client('lambda')
response = lambda_client.get_function(FunctionName="DetectModeration")
lambda_arn = response['Configuration']['FunctionArn']
#print (lambda_arn)
# 多行 JSON 内容
json_content = {
    "LambdaFunctionConfigurations": [
      {
        "LambdaFunctionArn": lambda_arn,
        "Events": ["s3:ObjectCreated:*"]
      }
    ]
}

# 将 JSON 内容写入文件
with open('s3triggerlambdaconfig.json', 'w') as f:
    json.dump(json_content, f)

如果需要加特定过滤器 使用以下代码

import json
import boto3

lambda_client = boto3.client('lambda')
response = lambda_client.get_function(FunctionName="DetectModeration")
lambda_arn = response['Configuration']['FunctionArn']
#print (lambda_arn)
# 多行 JSON 内容
json_content = {
    "LambdaFunctionConfigurations": [
      {
        "LambdaFunctionArn": lambda_arn,
        "Events": ["s3:ObjectCreated:*"],
        "Filter": {
          "Key": {
            "FilterRules": [
              {
                "Name": "prefix",
                "Value": "images"
              },
              {
                "Name": "suffix",
                "Value": ".jpg"
              }
            ]
          }
        }
      }
    ]
}

# 将 JSON 内容写入文件
with open('s3triggerlambdaconfig.json', 'w') as f:
    json.dump(json_content, f)

In [168]:
!aws s3api put-bucket-notification-configuration \
--bucket $bucketname \
--notification-configuration file://s3triggerlambdaconfig.json

In [ ]:
#查看日志

In [211]:
YouLogStreamName = "$(aws logs describe-log-streams --log-group-name /aws/lambda/DetectModeration --order-by LastEventTime --descending --query \'logStreams[0].logStreamName\' --output text)"

In [212]:
!aws logs get-log-events --log-group-name /aws/lambda/DetectModeration --log-stream-name '2023/07/26/[$LATEST]d7caa4fa9cf542b995e5b7116a7f3b83'

{
    "events": [
        {
            "timestamp": 1690340574141,
            "message": "INIT_START Runtime Version: python:3.10.v5\tRuntime Version ARN: arn:aws:lambda:ap-northeast-1::runtime:51b59a64d1fa5125d324f5fe77fbc805ea96f487f1d112fe4bf3f60323b552cb\n",
            "ingestionTime": 1690340579201
        },
        {
            "timestamp": 1690340574506,
            "message": "Loading function\n",
            "ingestionTime": 1690340579201
        },
        {
            "timestamp": 1690340574618,
            "message": "START RequestId: 996bd125-84db-4dd4-be8a-c0cf9306bf71 Version: $LATEST\n",
            "ingestionTime": 1690340579201
        },
        {
            "timestamp": 1690340575347,
            "message": "Detected labels for images/img4.jpeg\n",
            "ingestionTime": 1690340579201
        },
        {
            "timestamp": 1690340575364,
            "message": "[]\n",
            "ingestionTime": 1690340579201
        },
        {
            "ti